Scrape tweets for a period of 6 months. This has resulted in approximately 43.7K tweets.

In [1]:
import GetOldTweets3 as got
import pandas as pd
import warnings
import re
import numpy as np

# NTLK function
import nltk
from nltk.corpus import stopwords
from nltk import tokenize as tok
from nltk.stem.snowball import SnowballStemmer # load nltk's SnowballStemmer as variabled 'stemmer'
import lda # topic modeling -NMF & LDA
import string
from nltk.tag import StanfordNERTagger

warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.decomposition import LatentDirichletAllocation as LDA
# Tf-Idf and Clustering packages
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Text clustering with K-means and tf-idf
https://medium.com/@MSalnikov/text-clustering-with-k-means-and-tf-idf-f099bcf95183

In [2]:
search_terms = ['mortgage','current account','savings account','insurance','credit card','pension',
                'personal loan','money transfer','tax advice','investment','wealth management']

In [3]:
# scrape the data (terms)
tweet_df_all = pd.DataFrame()
for term in search_terms:
    print(term)
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(term)\
                                               .setSince("2019-04-13")\
                                               .setUntil("2019-09-28")\
                                               .setNear("Hong Kong")\
                                               .setWithin("310mi")
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    tweet_list = [[tweet[x].id,
                  tweet[x].author_id,
                  tweet[x].text,
                  tweet[x].retweets,
                  tweet[x].permalink,
                  tweet[x].date,
                  tweet[x].formatted_date,
                  tweet[x].favorites,
                  tweet[x].mentions,
                  tweet[x].hashtags,
                  tweet[x].geo,
                  tweet[x].urls
                 ]for x in range(0, len(tweet))]
    tweet_df = pd.DataFrame(tweet_list)
    tweet_df['search_term'] = term
    tweet_df_all = tweet_df_all.append(tweet_df)


tweet_df_all.columns = ['id','author_id','text','retweets','permalink','date','formatted_date','favorites','mentions','hashtags','geo','urls','search_term']

                                               

mortgage
current account
savings account
insurance
credit card
pension
personal loan


C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


money transfer
tax advice
investment
wealth management


In [4]:
tweet_df_all.to_csv(r"C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\data\all_tweet.csv", index=False)
tweet_df_all.shape

(385, 13)

Add List of Financial Institutions providng the above products

In [5]:
tweet_df_all = pd.read_csv(r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\data\all_tweet.csv')
tweet_df_all = tweet_df_all[tweet_df_all['text'].notna()]

In [6]:
print(tweet_df_all.shape);tweet_df_all.head()

(385, 13)


,id,author_id,text,retweets,permalink,date,formatted_date,favorites,mentions,hashtags,geo,urls,search_term
0,1177593827768422400,8.579987e+08,It is a free market so loans will exist. Howev...,0.0,https://twitter.com/liujackc/status/1177593827...,2019-09-27 14:40:21+00:00,Fri Sep 27 14:40:21 +0000 2019,3.0,NaN,NaN,NaN,NaN,mortgage
1,1169176926977581057,2.211808e+07,Lucky bastard! Actually I know someone who did...,0.0,https://twitter.com/Steve_Dunthorne/status/116...,2019-09-04 09:14:36+00:00,Wed Sep 04 09:14:36 +0000 2019,2.0,NaN,NaN,NaN,NaN,mortgage
2,1167796354032078850,2.566628e+09,Football historians will look back in astonish...,0.0,https://twitter.com/LysaghtSteph/status/116779...,2019-08-31 13:48:42+00:00,Sat Aug 31 13:48:42 +0000 2019,0.0,NaN,NaN,NaN,NaN,mortgage
3,1164907704894099462,5.383800e+07,#pirata #bravo #italian Best Italian food with...,0.0,https://twitter.com/raymoorecn/status/11649077...,2019-08-23 14:30:14+00:00,Fri Aug 23 14:30:14 +0000 2019,0.0,NaN,#pirata #bravo #italian,NaN,https://www.instagram.com/p/B1gojG0gLfRj02mhIX...,mortgage
4,1163704451305054208,2.162408e+09,Most HK Chinese were in that camp in 1997. Hat...,0.0,https://twitter.com/DrJim_Walker/status/116370...,2019-08-20 06:48:56+00:00,Tue Aug 20 06:48:56 +0000 2019,0.0,NaN,NaN,NaN,NaN,mortgage


In [7]:
tweet_df_all['text'] = tweet_df_all['text'].str.lower()
tweet_df_comp = tweet_df_all
tweet_df_comp.groupby('search_term')['id'].count()

search_term
credit card           42
current account        3
insurance             71
investment           216
money transfer         2
mortgage              18
pension               25
savings account        2
tax advice             1
wealth management      5
Name: id, dtype: int64

Topic Extraction with LDA model

In [8]:
#remove unnessary words
#Complie all regular expressions
isURL = re.compile(r'http[s]?:// (?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', re.VERBOSE | re.IGNORECASE)
isRTusername = re.compile(r'^RT+[\s]+(@[\w_]+:)',re.VERBOSE | re.IGNORECASE) #r'^RT+[\s]+(@[\w_]+:)'
isEntity = re.compile(r'@[\w_]+', re.VERBOSE | re.IGNORECASE)

# Helper functions
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])) 

# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
     
        
def clean_tweet(row):
    row = isURL.sub("",row)
    row = isRTusername.sub("",row)
    row = isEntity.sub("",row)
    return row

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in tok.sent_tokenize(text) for word in tok.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [9]:
tweet_df_comp['text_clean'] = tweet_df_comp['text'].apply(lambda row:clean_tweet(row))

#remove punctuations
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
tweet_df_comp['text_clean'] = tweet_df_comp['text_clean'].str.replace(RE_PUNCTUATION, "")
tweet_df_comp.head()

,id,author_id,text,retweets,permalink,date,formatted_date,favorites,mentions,hashtags,geo,urls,search_term,text_clean
0,1177593827768422400,8.579987e+08,it is a free market so loans will exist. howev...,0.0,https://twitter.com/liujackc/status/1177593827...,2019-09-27 14:40:21+00:00,Fri Sep 27 14:40:21 +0000 2019,3.0,NaN,NaN,NaN,NaN,mortgage,it is a free market so loans will exist howeve...
1,1169176926977581057,2.211808e+07,lucky bastard! actually i know someone who did...,0.0,https://twitter.com/Steve_Dunthorne/status/116...,2019-09-04 09:14:36+00:00,Wed Sep 04 09:14:36 +0000 2019,2.0,NaN,NaN,NaN,NaN,mortgage,lucky bastard actually i know someone who did ...
2,1167796354032078850,2.566628e+09,football historians will look back in astonish...,0.0,https://twitter.com/LysaghtSteph/status/116779...,2019-08-31 13:48:42+00:00,Sat Aug 31 13:48:42 +0000 2019,0.0,NaN,NaN,NaN,NaN,mortgage,football historians will look back in astonish...
3,1164907704894099462,5.383800e+07,#pirata #bravo #italian best italian food with...,0.0,https://twitter.com/raymoorecn/status/11649077...,2019-08-23 14:30:14+00:00,Fri Aug 23 14:30:14 +0000 2019,0.0,NaN,#pirata #bravo #italian,NaN,https://www.instagram.com/p/B1gojG0gLfRj02mhIX...,mortgage,pirata bravo italian best italian food without...
4,1163704451305054208,2.162408e+09,most hk chinese were in that camp in 1997. hat...,0.0,https://twitter.com/DrJim_Walker/status/116370...,2019-08-20 06:48:56+00:00,Tue Aug 20 06:48:56 +0000 2019,0.0,NaN,NaN,NaN,NaN,mortgage,most hk chinese were in that camp in 1997 hate...


In [10]:
# List of stopwords
import nltk
nltk.download('stopwords')

stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv(r"C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\pre_process\twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

# add in search terms as topic extraction is performed within each search topic, 
# we do not want the word or valriation of the word captured as a topic word
search_terms_revised = ['mortgages','wealthmanagement','pensions','money','transfer']
readInStopwords.extend(search_terms)
readInStopwords.extend(search_terms_revised)

stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# parameter for lda, i am selecrign 3 topic and 4 words for each of the search terms 
number_topics = 5
number_words = 5

In [12]:
tweets_all_topics= pd.DataFrame()
# term frequency modelling
for terms in tweet_df_comp['search_term'].unique():
    print(terms)
    tweets_search_topics  = tweet_df_comp[tweet_df_comp['search_term']==terms].reset_index(drop=True)
    corpus = tweets_search_topics['text_clean'].tolist()
    # print(corpus)
    tf_vectorizer = CountVectorizer(max_df=1, min_df=1, stop_words=stop_list, tokenizer=tokenize_only) 
    # Use tf (raw term count) features for LDA. !!!!!!!!!!!!!!
    tf = tf_vectorizer.fit_transform(corpus)
    
    # Create and fit the LDA model
    model = LDA(n_components=number_topics, n_jobs=-1)
    id_topic = model.fit(tf)
    # Print the topics found by the LDA model
    print("Topics found via LDA:")
    topic_keywords = show_topics(vectorizer=tf_vectorizer, lda_model=model, n_words=number_words)        
    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    df_topic_keywords = df_topic_keywords.reset_index()
    df_topic_keywords['topic_index'] = df_topic_keywords['index'].str.split(' ', n = 1, expand = True)[[1]].astype('int')
    print(df_topic_keywords)
    
    ############ get the dominat topic for each document in a data frame ###############
    # Create Document — Topic Matrix
    lda_output = model.transform(tf)
    # column names
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(corpus))]
    
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic   
    df_document_topic = df_document_topic.reset_index()
        
    #combine all the search terms into one data frame
    tweets_topics = tweets_search_topics.merge(df_document_topic, left_index=True, right_index=True, how='left')
    tweets_topics_words = tweets_topics.merge(df_topic_keywords, how='left', left_on='dominant_topic', right_on='topic_index')
    tweets_all_topics = tweets_all_topics.append(tweets_topics_words)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'account', 'across', 'advice', 'ai', 'almost', 'also', 'among', 'ca', 'card', 'credit', 'current', 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', 'however', 'least', 'let', 'like', 'likely', 'loan', 'management', 'may', 'might', 'must', "n't", 'need', 'neither', 'often', 'personal', 'rather', 'said', 'savings', 'say', 'says', 'sha', 'since', 'tax', 'tis', 'twas', 'us', 'wants', 'wealth', 'wo', 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


mortgage
Topics found via LDA:
     index  Word 0      Word 1    Word 2      Word 3    Word 4  topic_index
0  Topic 0   pogba  historians      join        look  nonsense            0
1  Topic 1     man        next  insisted    language   british            1
2  Topic 2     buy       china   italian  relatively   renting            2
3  Topic 3   think     feeling    normal        grow     whats            3
4  Topic 4  income        high      flat          mo   monthly            4
current account
Topics found via LDA:
     index   Word 0   Word 1        Word 2    Word 3    Word 4  topic_index
0  Topic 0    agree  whether  transactions   totally  possible            0
1  Topic 1  savings    rates   meaningless  relevant       low            1
2  Topic 2    agree  whether  transactions   totally  possible            2
3  Topic 3    agree  whether  transactions   totally  possible            3
4  Topic 4      new      hey          year      next   friends            4
savings account
Top

In [13]:
tweets_all_topics = tweets_all_topics.reset_index(drop=True)
print(tweets_all_topics.shape)
tweets_all_topics.head()

(385, 28)


,id,author_id,text,retweets,permalink,date,formatted_date,favorites,mentions,hashtags,...,Topic3,Topic4,dominant_topic,index_y,Word 0,Word 1,Word 2,Word 3,Word 4,topic_index
0,1177593827768422400,8.579987e+08,it is a free market so loans will exist. howev...,0.0,https://twitter.com/liujackc/status/1177593827...,2019-09-27 14:40:21+00:00,Fri Sep 27 14:40:21 +0000 2019,3.0,NaN,NaN,...,0.01,0.95,4,Topic 4,income,high,flat,mo,monthly,4
1,1169176926977581057,2.211808e+07,lucky bastard! actually i know someone who did...,0.0,https://twitter.com/Steve_Dunthorne/status/116...,2019-09-04 09:14:36+00:00,Wed Sep 04 09:14:36 +0000 2019,2.0,NaN,NaN,...,0.91,0.02,3,Topic 3,think,feeling,normal,grow,whats,3
2,1167796354032078850,2.566628e+09,football historians will look back in astonish...,0.0,https://twitter.com/LysaghtSteph/status/116779...,2019-08-31 13:48:42+00:00,Sat Aug 31 13:48:42 +0000 2019,0.0,NaN,NaN,...,0.01,0.01,0,Topic 0,pogba,historians,join,look,nonsense,0
3,1164907704894099462,5.383800e+07,#pirata #bravo #italian best italian food with...,0.0,https://twitter.com/raymoorecn/status/11649077...,2019-08-23 14:30:14+00:00,Fri Aug 23 14:30:14 +0000 2019,0.0,NaN,#pirata #bravo #italian,...,0.02,0.02,2,Topic 2,buy,china,italian,relatively,renting,2
4,1163704451305054208,2.162408e+09,most hk chinese were in that camp in 1997. hat...,0.0,https://twitter.com/DrJim_Walker/status/116370...,2019-08-20 06:48:56+00:00,Tue Aug 20 06:48:56 +0000 2019,0.0,NaN,NaN,...,0.01,0.01,1,Topic 1,man,next,insisted,language,british,1


In [14]:
tweets_all_topics.to_csv(r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\processed_data\tweets_all_topics.csv', index=False)

Sentiment analysis using Deep Learning

In [15]:
import keras
import h5py
from keras.models import model_from_json
from keras.models import load_model
import json
from nltk.tokenize import RegexpTokenizer

Using TensorFlow backend.


In [16]:
# read in the weight of the trained model.
weight_path = r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\models\dl_sentiment_model\best_weight_glove_bi_512.hdf5'
prd_model = load_model(weight_path)
prd_model.summary()
word_idx = json.load(open(r"C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\models\dl_sentiment_model\word_idx.txt"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\USER\Anaconda3\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 56, 300)           120000300 
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_20 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 120,576,310
Trainable params: 576,010
Non-trainable params: 120,000,300
_________________________________________________________________


In [17]:
def get_sentiment_DL(prd_model, text_data, word_idx):

    #data = "Pass the salt"

    live_list = []
    batchSize = len(text_data)
    live_list_np = np.zeros((56,batchSize))
    for index, row in text_data.iterrows():
        #print (index)
        text_data_sample = text_data['text'][index]
        # split the sentence into its words and remove any punctuations.
        tokenizer = RegexpTokenizer(r'\w+')
        text_data_list = tokenizer.tokenize(text_data_sample)

        #text_data_list = text_data_sample.split()


        labels = np.array(['1','2','3','4','5','6','7','8','9','10'], dtype = "int")
        #word_idx['I']
        # get index for the live stage
        data_index = np.array([word_idx[word.lower()] if word.lower() in word_idx else 0 for word in text_data_list])
        data_index_np = np.array(data_index)

        # padded with zeros of length 56 i.e maximum length
        padded_array = np.zeros(56)
        padded_array[:data_index_np.shape[0]] = data_index_np[:56]
        data_index_np_pad = padded_array.astype(int)


        live_list.append(data_index_np_pad)

    live_list_np = np.asarray(live_list)
    score = prd_model.predict(live_list_np, batch_size=batchSize, verbose=0)
    single_score = np.round(np.dot(score, labels)/10,decimals=2)

    score_all  = []
    for each_score in score:

        top_3_index = np.argsort(each_score)[-3:]
        top_3_scores = each_score[top_3_index]
        top_3_weights = top_3_scores/np.sum(top_3_scores)
        single_score_dot = np.round(np.dot(top_3_index, top_3_weights)/10, decimals = 2)
        score_all.append(single_score_dot)

    text_data['Sentiment_Score'] = pd.DataFrame(score_all)

    return text_data

In [18]:
text_data =  tweets_all_topics
# Deep Learning sentiment scoring
text_out = get_sentiment_DL(prd_model, text_data, word_idx)

#### example of negative tweets

In [19]:
text_out.sort_values(by='Sentiment_Score')[['text','Sentiment_Score']].head().T

,147,94,72,160,157
text,why is it always the answer to say the tories ...,when relationships ended i gave myself 24 hour...,oh yeah. you’re right. damn the pesky eu good ...,a. not sure why you are comparing labour to to...,that’s completely untrue. i was disgusted at l...
Sentiment_Score,0.07,0.08,0.09,0.09,0.1


####  example of positive tweets

In [20]:
text_out.sort_values(by='Sentiment_Score', ascending=False)[['text','Sentiment_Score']].head().T

,193,366,207,230,254
text,"""one of the finest clos apaltas of all time,"" ...","one of the great pessac léognans of 2014, the ...","one of the greatest salons ever released, the ...",one of the most impressive international colla...,"one of the greatest producers in portugal, tay..."
Sentiment_Score,0.86,0.85,0.84,0.83,0.83


In [21]:
text_out.to_csv(r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\processed_data\tweets_topics_sentiment.csv', index=False)

### Named Entity Recognition

In [22]:
def get_NER(text_data):
    #/Users/prajwalshreyas/Desktop/Singularity/dockerApps/ner-algo/stanford-ner-2015-01-30
    stanford_classifier = r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\models\ner\english.all.3class.distsim.crf.ser.gz'
    stanford_ner_path = r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\models\ner\stanford-ner.jar'

    
    st = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-16')
    
    print ('start get_NER')
    text_out = text_data.copy()
    doc = [ docs + ' 12345678 ' for docs in list(text_data['text'])]
    # ------------------------- Stanford Named Entity Recognition
    tokens = nltk.word_tokenize(str(doc))
    entities = st.tag(tokens) # actual tagging takes place using Stanford NER algorithm


    entities = [list(elem) for elem in entities] # Convert list of tuples to list of list
    print ('tag complete')
    for idx,element in enumerate(entities):
        try:
            if entities[idx][0] == '12345678':
                entities[idx][1] = "DOC_NUMBER"  #  Modify data by adding the tag "Doc_Number"
            #elif entities[idx][0].lower() == keyword:
            #    entities[idx][1] = "KEYWORD"
            # Combine First and Last name into a single word
            elif entities[idx][1] == "PERSON" and entities[idx + 1][1] == "PERSON":
                entities[idx + 1][0] = entities[idx][0] + '-' + entities[idx+1][0]
                entities[idx][1] = 'Combined'
            # Combine consecutive Organization names
            elif entities[idx][1] == 'ORGANIZATION' and entities[idx + 1][1] == 'ORGANIZATION':
                entities[idx + 1][0] = entities[idx][0] + '-' + entities[idx+1][0]
                entities[idx][1] = 'Combined'
        except IndexError:
            break
    print ('enumerate complete')
    # Filter list of list for the words we are interested in
    filter_list = ['DOC_NUMBER','PERSON','LOCATION','ORGANIZATION']
    entityWordList = [element for element in entities if any(i in element for i in filter_list)]

    entityString = ' '.join(str(word) for insideList in entityWordList for word in insideList) # convert list to string and concatenate it
    entitySubString = entityString.split("DOC_NUMBER") # split the string using the separator 'TWEET_NUMBER'
    del entitySubString[-1] # delete the extra blank row created in the previous step

    # Store the classified NERs in the main tweet data frame
    for idx,docNER in enumerate(entitySubString):
        docNER = docNER.strip().split() # split the string into word list
        # Filter for words tagged as Organization and store it in data frame
        text_out.loc[idx,'Organization'] =  ','.join([docNER[i-1]  for i,x in enumerate(docNER) if x == 'ORGANIZATION'])
        # Filter for words tagged as LOCATION and store it in data frame
        text_out.loc[idx,'Place'] = ','.join([docNER[i-1] for i,x in enumerate(docNER) if x == 'LOCATION'])
        # Filter for words tagged as PERSON and store it in data frame
        text_out.loc[idx,'Person'] = ','.join([docNER[i-1]  for i,x in enumerate(docNER) if x == 'PERSON'])

    print ('process complete')
    return text_out

In [23]:
text_ner_out = get_NER(text_out)

start get_NER
tag complete
enumerate complete
process complete


In [30]:
print(text_ner_out.head())

                    id     author_id  \
0  1177593827768422400  8.579987e+08   
1  1169176926977581057  2.211808e+07   
2  1167796354032078850  2.566628e+09   
3  1164907704894099462  5.383800e+07   
4  1163704451305054208  2.162408e+09   

                                                text  retweets  \
0  it is a free market so loans will exist. howev...       0.0   
1  lucky bastard! actually i know someone who did...       0.0   
2  football historians will look back in astonish...       0.0   
3  #pirata #bravo #italian best italian food with...       0.0   
4  most hk chinese were in that camp in 1997. hat...       0.0   

                                           permalink  \
0  https://twitter.com/liujackc/status/1177593827...   
1  https://twitter.com/Steve_Dunthorne/status/116...   
2  https://twitter.com/LysaghtSteph/status/116779...   
3  https://twitter.com/raymoorecn/status/11649077...   
4  https://twitter.com/DrJim_Walker/status/116370...   

                        d

#### output  the file 

In [31]:
text_ner_out.to_csv(r'C:\Users\USER\Desktop\twitter-sentiment-stock-prediction\processed_data\tweets_topics_sentiment_ner.csv', index=False)